Assignment 3 Mikołaj Łuczak

importing modules and custom DataSet

In [ ]:
import os
from os import listdir
import os.path
from os.path import isfile
from os.path import join
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transform
from skimage import io
import PIL
from sklearn.model_selection import train_test_split as tts
import torch.nn as nn

path = "./data"
folder = ['0', '1', '2', '3']

class CustomDataset(Dataset):
    def __init__(self, path, folders, test=True,  transform=None):
        self.dataPath = path
        self.folders = folders
        self.labels = []
        self.flist = []
        self.transform = transform
        for f in folders:
            path1 = self.dataPath+"/"+f
            files = [path1+"/"+f for f in listdir(path1) if isfile(join(path1, f))]
            self.flist.extend(files)
            self.labels.extend([f]*len(files))
        self.test = test
        x_train, x_test, y_train, y_test = tts(self.flist, self.labels, test_size=0.33)
        if test:
            self.flist = x_test
            self.labels = y_test
        else:
            self.flist = x_train
            self.labels = y_train

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        label = np.asarray(self.labels[index], dtype=np.compat.long)
        filename = self.flist[index]
        file1 = PIL.Image.open(filename)

        if self.transform:
            file = self.transform(file1)

        return file, label

Creating Training Dataloader and testing Dataloader

In [ ]:
CustomDS = CustomDataset(path, folder, transform=transform.Compose([transform.ToTensor()]))

#train_loader = DataLoader(CustomDS, batch_size=64,shuffle=True, num_workers=0, pin_memory=False, persistent_workers=False)
train_loader = DataLoader(CustomDS, batch_size=64, shuffle=True)

CustomDS = CustomDataset(path, folder, test=True,  transform=transform.Compose([transform.ToTensor()]))

test_loader = DataLoader(CustomDS, batch_size=64, shuffle=True)
#test_loader = DataLoader(CustomDS, batch_size=32, shuffle=True, num_workers=0, pin_memory=False, persistent_workers=False)

Custom Neural Network

In [ ]:
class CNN(nn.Module):
  def __init__(self, input_depth, num_classes):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(input_depth, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv3 = nn.Conv2d(20, 20, kernel_size=3)
    self.conv4 = nn.Conv2d(20, 30, kernel_size=3)

    self.fc1 = nn.Linear(30*1*1, 30)
    self.fc2 = nn.Linear(30, num_classes)
    self.activation = nn.ReLU()
    self.downsample = nn.MaxPool2d(2)

  def forward(self, data):
    output = self.downsample(self.activation(self.conv1(data)))
    output = self.downsample(self.activation(self.conv2(output)))
    output = self.activation(self.conv3(output))
    output = self.activation(self.conv4(output))

    output = nn.functional.adaptive_avg_pool2d(output, (1,1))

    output = output.view(-1, output.shape[1]*output.shape[2]*output.shape[3])

    output = self.activation(self.fc1(output))
    output = self.fc2(output)

    return output

Setting Device, Creating model optimazer and loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN(3, 4).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()

Training and Testing Functions

In [ ]:
def train(epoch, data):
    model.train()

    for id, (images, target) in enumerate(data):
      images, target = images.to(device), target.to(device)
      target=target.long()

      optimizer.zero_grad()

      output = model(images)
      # print(output1.shape)

      error = (loss(output, target))

      error.backward()
      optimizer.step()

      if id % 10 == 0:
        print("Training error %.3f"%(error.data.item()))


def test(data):
  model.eval()
  correct=0

  for batch_id, (images, target) in enumerate(data):
      images, target = images.to(device), target.to(device)
      target = target.long()

      with torch.no_grad():
        output = model(images)

        predicted = output.data.max(1)[1]

        correct+=predicted.eq(target.data).cpu().sum()

  print("Accuarcy: %3f"%(correct/len(data.dataset)))

  return output

Calling out a train and test function

In [ ]:
#epochs = 100
epochs = 10
for i in range(epochs):
    train(1, train_loader)

test(test_loader)

I had some problems with this assignment but I finally manage to do it.


I am very sorry for the delay